In [2]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from scipy.sparse import hstack

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/dataset/r_d.xlsx')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
# Remove leading and trailing spaces from the 'Opening Hours' column
df['Opening Hours'] = df['Opening Hours'].str.strip()

# Remove any additional unconverted data, such as '- 12:00 AM'
df['Opening Hours'] = df['Opening Hours'].str.replace('-.*', '', regex=True)

# Convert to datetime format
df['Opening Hours'] = pd.to_datetime(df['Opening Hours'], format='%I:%M %p')

In [ ]:
df.info()

In [ ]:
# @title City vs Rating
figsize = (12, 1.2 * len(df['City'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(df, x='Rating', y='City', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [ ]:
df.groupby('City').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
# @title Rating
df['Rating'].plot(kind='hist', bins=20, title='Rating')
plt.gca().spines[['top', 'right',]].set_visible(False)

---
# Resturant Recomendation
---

In [ ]:
# vectorizer for cuisine
cuisine_tfidf = TfidfVectorizer()
cuisine_feature = cuisine_tfidf.fit_transform(df['Cuisine'])

In [ ]:
# vectorizer for location
location_tfidf = TfidfVectorizer()
location_feature = location_tfidf.fit_transform(df['Location'])

In [ ]:

# Combine the feature vectors
combined_features = hstack([cuisine_feature, location_feature])

In [ ]:
# Compute the cosine similarity matrix
cosine_similarities = linear_kernel(combined_features, combined_features)

In [ ]:
# Function to get restaurant recommendations based on a given restaurant name
def get_recommendations(name, cosine_similarities, df, top_n=5):
    # Get the index of the restaurant
    idx = df[df['Name'] == name].index[0]

    # Get the pairwise similarities of the restaurant with other restaurants
    sim_scores = list(enumerate(cosine_similarities[idx]))

    # Sort the restaurants based on similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the top N similar restaurants (excluding the input restaurant itself)
    top_similar_restaurants = [sim_scores[i][0] for i in range(1, top_n + 1)]

    # Return the recommended restaurant names
    return df['Name'].iloc[top_similar_restaurants]


In [ ]:
# Get recommendations for a restaurant
restaurant_name = 'Curry House'
recommendations = get_recommendations(restaurant_name, cosine_similarities, df)

print(f"Recommendations for {restaurant_name}:")
print(recommendations)

In [ ]:
import pickle

# Assuming you have trained and evaluated your model
model = 'recommendations'  # Your trained model object

# Save the model to disk using pickle
with open('recommended_model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [ ]:
! streamlit rum app.py & npx localtunnel --port 8501

In [ ]:
!wget -q -O - ipv4.icanhazip.com

In [ ]:
!pip install streamlit -q